In [1]:
import re
from vnpy.trader.constant import Exchange
from vnpy.app.cta_strategy.base import EXCHANGE_SYMBOL_DICT
from datetime import datetime, timedelta

In [36]:
fromDate = datetime.strptime('2012-6-10', '%Y-%m-%d')
toDate = datetime.strptime('2017-12-01', '%Y-%m-%d')

count = 0
while fromDate.date() != toDate.date():
    count += 1
    fromDate = fromDate + timedelta(1)

print(count)

2000


In [16]:
# 当前时间戳
import time
a = int(time.time())
print(a)

1563864223


In [59]:
# 时间戳转时间
timeStamp = 1512136800
date = datetime.fromtimestamp(timeStamp)
print(date)

2017-12-01 22:00:00


In [23]:
# 时间转时间戳
a = '2019-07-20 08:00:00'
b = datetime.strptime(a, '%Y-%m-%d %H:%M:%S')
print(b.timestamp())

1563580800.0


In [4]:
import requests
URL_EXCHANGES = 'https://www.cryptocompare.com/api/data/exchanges'
response = requests.get(URL_EXCHANGES).json()
data = response['Data']
print(data.keys())

dict_keys(['Ethfinex', 'DSX', 'Coinnest', 'IDEX', 'Bitlish', 'CoinEx', 'Buda', 'CoinsBank', 'Braziliex', 'Upbit', 'Neraex', 'BTCTurk', 'BitMart', 'Bibox', 'DDEX', 'ZB', 'LBank', 'SingularityX', 'Simex', 'Nebula', 'Bluebelt', 'RightBTC', 'WorldCryptoCap', 'ABCC', 'OpenLedger', 'WEX', 'BitexBook', 'CryptoCarbon', 'BigONE', 'CoinBene', 'FCoin', 'Kuna', 'Tokenomy', 'IDAX', 'CoinHub', 'Nexchange', 'Hikenex', 'Qryptos', 'BTCBOX', 'HADAX', 'CryptoBulls', 'IndependentReserve', 'Bitmex', 'DEx', 'Ironex', 'Zecoex', 'Nlexch', 'Globitex', 'Ore', 'Bitinfi', 'CoinJar', 'TDAX', 'CoinFalcon', 'ACX', 'Bitsane', 'BTCAlpha', 'FCCE', 'InstantBitex', 'Gneiss', 'Cryptonit', 'CoinPulse', 'Bitforex', 'Liqnet', 'BCEX', 'Liquid', 'Minebit', 'IQFinex', 'P2PB2B', 'StocksExchange', 'iCoinbay', 'CoinTiger', 'Threexbit', 'cobinhood', 'Switcheo', 'Bitkub', 'Everbloom', 'EXRATES', 'Coinsbit', 'NDAX', 'DigiFinex', 'Bitshares', 'Coinmate', 'Incorex', 'Ethermium', 'Nuex', 'Blackturtle', 'Catex', 'Exenium', 'StocksExchang

In [5]:
a = 'OKEX/ETH.USDT'
b = a.split('/')[-1]
c = b.split('.')[0]
print(a)
print(b)
print(c)

OKEX/ETH.USDT
ETH.USDT
ETH


In [ ]:
Poloniex
HitBTC